# NOAA : Extraction de la pluviométrie

Dataset :
- Dossier /demo/noaa/noaa-raw-data
- Format du fichier : "STATION","DATE","LATITUDE","LONGITUDE","ELEVATION","NAME","PRCP","PRCP_ATTRIBUTES"
- Une ligne par relevé météo par jour

Exercice :
- Récupérer le code pays sur 2 lettres et la placer dans une nouvelle colonne "COUNTRY"
- Supprimer les lignes où PRCP est null
- Regrouper les relevés des stations par pays
- Calculer la moyenne de pluie ("PRCP"), la somme, le maximum, l'écart type, le nombre de stations météo
- Trier par date et par pays
- Sauver dans '/stagiaire/votre_dossier/noaa/daily_rain_by_country'

In [67]:
%%writefile spark_noaa.py
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = SparkSession \
    .builder \
    .appName('Spark RDD') \
    .getOrCreate()
sc = spark.sparkContext
lines = spark.read.text('/demo/noaa/noaa-raw-data/FRW00034067.csv') 
def attributes(ligne):
    station_name = ligne[0].replace('"', '')
    return [station_name] + ligne[7].split(',')
mon_rdd=lines.rdd.map(lambda r: r[0].split('","')).map(attributes).filter(lambda x: x[1]!='PRCP_ATTRIBUTES')
mon_rdd.take(2)
sc.stop()

Overwriting spark_noaa.py


In [64]:
#!hdfs dfs -ls /demo/noaa/noaa-raw-data

In [1]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
DATASET_FOLDER = '/demo/noaa/noaa-raw-data'
SPARK_MASTER = 'spark://localhost:7077'
APP_NAME = 'Group daily rain by Country - 430'

noaa_csv_path = DATASET_FOLDER 
output = '/stagiaire/votre_dossier/noaa/daily_rain_by_country'

# Create Spark session
spark = SparkSession.builder.master(SPARK_MASTER).appName(APP_NAME).getOrCreate()


    Récupérer le code pays sur 2 lettres et la placer dans une nouvelle colonne "COUNTRY"
    Supprimer les lignes où PRCP est null
    Regrouper les relevés des stations par pays
    Calculer la moyenne de pluie ("PRCP"), la somme, le maximum, l'écart type, le nombre de stations météo
    Trier par date et par pays
    Sauver dans '/stagiaire/votre_dossier/noaa/daily_rain_by_country'


In [4]:
# Utiliser RDD pour lire le fichier et extraire le code pays, la date et la température

In [62]:
#lines = spark.read.text('/demo/noaa/noaa-raw-data/FRW00034067.csv') 
#lines = spark.read.text('/demo/noaa/noaa-raw-data/FRE00104040.csv') 

In [63]:
lines.rdd.map(lambda r: r).take(3)

[Row(value='"STATION","DATE","LATITUDE","LONGITUDE","ELEVATION","NAME","PRCP","PRCP_ATTRIBUTES","SNOW","SNOW_ATTRIBUTES","SNWD","SNWD_ATTRIBUTES","TMAX","TMAX_ATTRIBUTES","TMIN","TMIN_ATTRIBUTES","WSFG","WSFG_ATTRIBUTES","WT01","WT01_ATTRIBUTES","WT03","WT03_ATTRIBUTES","WT05","WT05_ATTRIBUTES","WT08","WT08_ATTRIBUTES","WT16","WT16_ATTRIBUTES","WT18","WT18_ATTRIBUTES"'),
 Row(value='"FRW00034067","1958-09-01","48.43333","4.48333","120.1","BRIENNE, FR","   25",",,X","    0",",,X","    0",",,X","  200",",,X","  183",",,X",,,"    1",",,X","    1",",,X",,,,,"    1",",,X",,'),
 Row(value='"FRW00034067","1958-09-02","48.43333","4.48333","120.1","BRIENNE, FR","    0",",,X","    0",",,X","    0",",,X","  244",",,X","  172",",,X",,,"    1",",,X",,,,,"    1",",,X",,,,')]

In [6]:
lines.rdd.map(lambda r: r).take(3)

[Row(value='"STATION","DATE","LATITUDE","LONGITUDE","ELEVATION","NAME","PRCP","PRCP_ATTRIBUTES","SNOW","SNOW_ATTRIBUTES","SNWD","SNWD_ATTRIBUTES","TMAX","TMAX_ATTRIBUTES","TMIN","TMIN_ATTRIBUTES","WSFG","WSFG_ATTRIBUTES","WT01","WT01_ATTRIBUTES","WT03","WT03_ATTRIBUTES","WT05","WT05_ATTRIBUTES","WT08","WT08_ATTRIBUTES","WT16","WT16_ATTRIBUTES","WT18","WT18_ATTRIBUTES"'),
 Row(value='"FRW00034067","1958-09-01","48.43333","4.48333","120.1","BRIENNE, FR","   25",",,X","    0",",,X","    0",",,X","  200",",,X","  183",",,X",,,"    1",",,X","    1",",,X",,,,,"    1",",,X",,'),
 Row(value='"FRW00034067","1958-09-02","48.43333","4.48333","120.1","BRIENNE, FR","    0",",,X","    0",",,X","    0",",,X","  244",",,X","  172",",,X",,,"    1",",,X",,,,,"    1",",,X",,,,')]

In [22]:
lines.rdd.map(lambda r: r[0]).take(3)

['"STATION","DATE","LATITUDE","LONGITUDE","ELEVATION","NAME","PRCP","PRCP_ATTRIBUTES","SNOW","SNOW_ATTRIBUTES","SNWD","SNWD_ATTRIBUTES","TMAX","TMAX_ATTRIBUTES","TMIN","TMIN_ATTRIBUTES","WSFG","WSFG_ATTRIBUTES","WT01","WT01_ATTRIBUTES","WT03","WT03_ATTRIBUTES","WT05","WT05_ATTRIBUTES","WT08","WT08_ATTRIBUTES","WT16","WT16_ATTRIBUTES","WT18","WT18_ATTRIBUTES"',
 '"FRW00034067","1958-09-01","48.43333","4.48333","120.1","BRIENNE, FR","   25",",,X","    0",",,X","    0",",,X","  200",",,X","  183",",,X",,,"    1",",,X","    1",",,X",,,,,"    1",",,X",,',
 '"FRW00034067","1958-09-02","48.43333","4.48333","120.1","BRIENNE, FR","    0",",,X","    0",",,X","    0",",,X","  244",",,X","  172",",,X",,,"    1",",,X",,,,,"    1",",,X",,,,']

In [23]:
lines.rdd.map(lambda r: r[0][:5] ).take(3)

['"STAT', '"FRW0', '"FRW0']

In [27]:
lines.rdd.map(lambda r: (r[0][:5],r[0][5:10]) ).take(3)

[('"STAT', 'ION",'), ('"FRW0', '00340'), ('"FRW0', '00340')]

In [36]:
lines.rdd.map(lambda r: (r[0][:5],r[0][5:10]) ).map(lambda x: x[0] + x[1]).take(3)

['"STATION",', '"FRW000340', '"FRW000340']

In [37]:
lines.rdd.map(lambda r: (r[0][:5],r[0][5:10]) ).map(lambda x: x[0] + x[1]).count()

635

In [40]:
lines.rdd.map(lambda r: (r[0][:5],r[0][5:10]) ).map(lambda x: x[0] + x[1]).flatMap(lambda r: r).count()

6350

In [7]:
lines.rdd.map(lambda r: (r[0][:5],r[0][5:10]) ).map(lambda x: x[0] + x[1]).flatMap(lambda r: r).take(3)

['"', 'S', 'T']

In [11]:
lines.rdd.map(lambda r: r[0].split(',')).take(1)

[['"STATION"',
  '"DATE"',
  '"LATITUDE"',
  '"LONGITUDE"',
  '"ELEVATION"',
  '"NAME"',
  '"PRCP"',
  '"PRCP_ATTRIBUTES"',
  '"SNOW"',
  '"SNOW_ATTRIBUTES"',
  '"SNWD"',
  '"SNWD_ATTRIBUTES"',
  '"TMAX"',
  '"TMAX_ATTRIBUTES"',
  '"TMIN"',
  '"TMIN_ATTRIBUTES"',
  '"WSFG"',
  '"WSFG_ATTRIBUTES"',
  '"WT01"',
  '"WT01_ATTRIBUTES"',
  '"WT03"',
  '"WT03_ATTRIBUTES"',
  '"WT05"',
  '"WT05_ATTRIBUTES"',
  '"WT08"',
  '"WT08_ATTRIBUTES"',
  '"WT16"',
  '"WT16_ATTRIBUTES"',
  '"WT18"',
  '"WT18_ATTRIBUTES"']]

In [13]:
lines.rdd.map(lambda r: r[0].split(',')).count()

635

In [12]:
lines.rdd.map(lambda r: r[0].split(',')).flatMap(lambda r: r).take(1)

['"STATION"']

In [14]:
lines.rdd.map(lambda r: r[0].split(',')).flatMap(lambda r: r).count()

22302

In [20]:
lines.rdd.map(lambda r: r[0].split(',')).map(lambda x: [x[0].replace('"',''), x[3].replace('"','')]).take(5)

[['STATION', 'LONGITUDE'],
 ['FRW00034067', '4.48333'],
 ['FRW00034067', '4.48333'],
 ['FRW00034067', '4.48333'],
 ['FRW00034067', '4.48333']]

# Exercice
Récupérer la colonne PRCP_ATTRIBUTES et la découper pour obtenir 3 colonnes

In [30]:
mon_rdd = lines.rdd.map(lambda r: r[0].split('","'))
mon_rdd.take(1)

[['"STATION',
  'DATE',
  'LATITUDE',
  'LONGITUDE',
  'ELEVATION',
  'NAME',
  'PRCP',
  'PRCP_ATTRIBUTES',
  'SNOW',
  'SNOW_ATTRIBUTES',
  'SNWD',
  'SNWD_ATTRIBUTES',
  'TMAX',
  'TMAX_ATTRIBUTES',
  'TMIN',
  'TMIN_ATTRIBUTES',
  'WSFG',
  'WSFG_ATTRIBUTES',
  'WT01',
  'WT01_ATTRIBUTES',
  'WT03',
  'WT03_ATTRIBUTES',
  'WT05',
  'WT05_ATTRIBUTES',
  'WT08',
  'WT08_ATTRIBUTES',
  'WT16',
  'WT16_ATTRIBUTES',
  'WT18',
  'WT18_ATTRIBUTES"']]

In [35]:
mon_rdd=lines.rdd.map(lambda r: r[0].split('","')).map(lambda x: x[7].split(','))
mon_rdd.take(2)

[['PRCP_ATTRIBUTES'], ['', '', 'X']]

In [40]:
['a'] + ['b']

['a', 'b']

In [44]:
mon_rdd=lines.rdd.map(lambda r: r[0].split('","')).map(lambda x: x[7].split(',')).filter(lambda x: x[0]!='PRCP_ATTRIBUTES')
mon_rdd.take(2)

[['', '', 'X'], ['', '', 'X']]

In [57]:
def attributes(ligne):
    station_name = ligne[0].replace('"', '')
    return [station_name] + ligne[7].split(',')

In [58]:
mon_rdd=lines.rdd.map(lambda r: r[0].split('","')).map(attributes).filter(lambda x: x[1]!='PRCP_ATTRIBUTES')
mon_rdd.take(2)

[['FRW00034067', '', '', 'X'], ['FRW00034067', '', '', 'X']]

In [59]:
df = spark.createDataFrame(mon_rdd).toDF('STATION','col1', 'col2', 'col3')
df.show(3)

+-----------+----+----+----+
|    STATION|col1|col2|col3|
+-----------+----+----+----+
|FRW00034067|    |    |   X|
|FRW00034067|    |    |   X|
|FRW00034067|    |    |   X|
+-----------+----+----+----+
only showing top 3 rows



In [29]:
lines.rdd.map(lambda r: (r[0][:5],r[0][5:10]) ).flatMap(lambda r: r).take(10)

['"STAT',
 'ION",',
 '"FRW0',
 '00340',
 '"FRW0',
 '00340',
 '"FRW0',
 '00340',
 '"FRW0',
 '00340']

In [20]:
lines.rdd.flatMap(lambda r: r).take(10)

['"STATION","DATE","LATITUDE","LONGITUDE","ELEVATION","NAME","PRCP","PRCP_ATTRIBUTES","SNOW","SNOW_ATTRIBUTES","SNWD","SNWD_ATTRIBUTES","TMAX","TMAX_ATTRIBUTES","TMIN","TMIN_ATTRIBUTES","WSFG","WSFG_ATTRIBUTES","WT01","WT01_ATTRIBUTES","WT03","WT03_ATTRIBUTES","WT05","WT05_ATTRIBUTES","WT08","WT08_ATTRIBUTES","WT16","WT16_ATTRIBUTES","WT18","WT18_ATTRIBUTES"',
 '"FRW00034067","1958-09-01","48.43333","4.48333","120.1","BRIENNE, FR","   25",",,X","    0",",,X","    0",",,X","  200",",,X","  183",",,X",,,"    1",",,X","    1",",,X",,,,,"    1",",,X",,',
 '"FRW00034067","1958-09-02","48.43333","4.48333","120.1","BRIENNE, FR","    0",",,X","    0",",,X","    0",",,X","  244",",,X","  172",",,X",,,"    1",",,X",,,,,"    1",",,X",,,,',
 '"FRW00034067","1958-09-03","48.43333","4.48333","120.1","BRIENNE, FR","    0",",,X","    0",",,X","    0",",,X","  278",",,X","  150",",,X",,,"    1",",,X",,,,,"    1",",,X",,,,',
 '"FRW00034067","1958-09-04","48.43333","4.48333","120.1","BRIENNE, FR","    0"

In [ ]:
df.write.csv(output, header=True)
print(df.columns)

In [ ]:
sc.stop()